# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extension of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3.6
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0707 15:26:28.334082 4768265664 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0707 15:26:28.354856 4768265664 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [23]:
bob = sy.VirtualWorker(hook, id="bob")

In [24]:
bob._objects

{}

In [25]:
x = th.tensor([1,2,3,4,5])

In [26]:
# Send the tensor to bob
x = x.send(bob)

In [27]:
bob._objects

{93866940726: tensor([1, 2, 3, 4, 5])}

In [28]:
x.location

<VirtualWorker id:bob #objects:1>

In [29]:
x.id_at_location

93866940726

In [30]:
# This to get the id of the local machine (me)
x.id

52797135077

In [31]:
x.owner

<VirtualWorker id:me #objects:0>

In [32]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [33]:
x

(Wrapper)>[PointerTensor | me:52797135077 -> bob:93866940726]

In [34]:
# Get the tensor from bob and freeing the resource : pointer
x = x.get()

In [35]:
x

tensor([1, 2, 3, 4, 5])

In [36]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [37]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id='alice')

In [38]:
bob._objects

{}

In [39]:
alice._objects

{}

In [40]:
x = th.tensor([1,2,3,4,5])

In [49]:
x = x.send(bob,alice)

In [50]:
x

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:87543785747 -> bob:78112306649]
	-> (Wrapper)>[PointerTensor | me:99830694167 -> alice:88932022235]

In [51]:
x = x.get()

In [52]:
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [54]:
bob._objects, alice._objects

({}, {})

In [71]:
y = th.tensor([6,7,8,9,10])
y = y.send(bob,alice)

In [73]:
y.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:92256331573 -> bob:61210627182], 'alice': (Wrapper)>[PointerTensor | me:18039845090 -> alice:85905307780]}

In [69]:
# MultiPointerTensor
y

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:61556607052 -> bob:20483480976]
	-> (Wrapper)>[PointerTensor | me:69131552386 -> alice:59057926480]

In [70]:
y = y.get()

In [74]:
y.get(sum_results=True)

tensor([12, 14, 16, 18, 20])

# Lesson: Introducing Remote Arithmetic

In [75]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [76]:
x

(Wrapper)>[PointerTensor | me:81075738780 -> bob:30849624082]

In [77]:
y

(Wrapper)>[PointerTensor | me:33184930411 -> bob:44064299207]

In [78]:
z = x + y

In [79]:
z

(Wrapper)>[PointerTensor | me:1539670676 -> bob:3931108259]

In [80]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [81]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:12265641437 -> bob:2704249267]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [82]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [83]:
z = (x + y).sum()

In [84]:
# we can apply to it back propagation 
z.backward()

(Wrapper)>[PointerTensor | me:88506756706 -> bob:8605706381]

In [85]:
x = x.get()

In [86]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [87]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [88]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [89]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [110]:
bob = bob.clear_objects()

In [111]:
bob._objects

{}

In [112]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [114]:
x

(Wrapper)>[PointerTensor | me:58488243854 -> bob:9451694665]

In [115]:
bob._objects

{}

In [116]:
del x

In [117]:
x

NameError: name 'x' is not defined

In [118]:
bob._objects

{}

In [119]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [120]:
bob._objects

{}

In [121]:
x = "asdf"

In [122]:
bob._objects

{}

In [123]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [124]:
x

(Wrapper)>[PointerTensor | me:28180650759 -> bob:26751647266]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [125]:
bob._objects

{}

In [126]:
del x

In [127]:
bob._objects

{}

In [128]:
bob = bob.clear_objects()
bob._objects

{}

In [129]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [130]:
bob._objects

{}

In [131]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [132]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:74187076482 -> bob:76315476733]
Tensor B: tensor([1, 1, 1, 1, 1])

In [133]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [134]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:0> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [1]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

W0708 17:12:10.725677 4516472256 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0708 17:12:10.747699 4516472256 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
from torch import nn, optim

In [3]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [4]:
# A Toy Model
model = nn.Linear(2,1)

In [5]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [6]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(9.6193)
tensor(0.9932)
tensor(0.2714)
tensor(0.1651)
tensor(0.1194)
tensor(0.0885)
tensor(0.0660)
tensor(0.0494)
tensor(0.0371)
tensor(0.0279)
tensor(0.0210)
tensor(0.0159)
tensor(0.0121)
tensor(0.0091)
tensor(0.0069)
tensor(0.0053)
tensor(0.0040)
tensor(0.0031)
tensor(0.0023)
tensor(0.0018)


In [7]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [8]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [9]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [10]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # ! send model to the data !
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [11]:
train()

tensor(2.3718, requires_grad=True)
tensor(0.0115, requires_grad=True)
tensor(0.0399, requires_grad=True)
tensor(0.0185, requires_grad=True)
tensor(0.0121, requires_grad=True)
tensor(0.0113, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0041, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(9.6011e-05, requires_grad=True)
tensor(7.3383e-05, requires_grad=True)
tensor(6.3639e-05, requires_grad=True)
tensor(4.9034e-05, requires_grad=True)
tens

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [12]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [14]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [15]:
x = x.send(alice)

In [16]:
bob._objects

{52302763536: tensor([1, 2, 3, 4, 5])}

In [17]:
alice._objects

{45464412481: (Wrapper)>[PointerTensor | alice:45464412481 -> bob:52302763536]}

In [18]:
y = x + x

In [19]:
y

(Wrapper)>[PointerTensor | me:48245103328 -> alice:79603568093]

In [20]:
bob._objects

{52302763536: tensor([1, 2, 3, 4, 5]),
 52154218474: tensor([ 2,  4,  6,  8, 10])}

In [21]:
alice._objects

{45464412481: (Wrapper)>[PointerTensor | alice:45464412481 -> bob:52302763536],
 79603568093: (Wrapper)>[PointerTensor | alice:79603568093 -> bob:52154218474]}

In [22]:
jon = sy.VirtualWorker(hook, id="jon")

In [23]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [24]:
bob._objects

{53958391141: tensor([1, 2, 3, 4, 5])}

In [25]:
alice._objects

{92007380369: (Wrapper)>[PointerTensor | alice:92007380369 -> bob:53958391141]}

In [26]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:92007380369 -> bob:53958391141]

In [27]:
bob._objects

{53958391141: tensor([1, 2, 3, 4, 5])}

In [28]:
alice._objects

{}

In [29]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [30]:
bob._objects

{}

## Garbage collection 

In [32]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [33]:
bob._objects

{16156287205: tensor([1, 2, 3, 4, 5])}

In [34]:
alice._objects

{72840854718: (Wrapper)>[PointerTensor | alice:72840854718 -> bob:16156287205]}

In [35]:
del x

In [36]:
bob._objects

{}

In [37]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [59]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [39]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [40]:
bob._objects

{64230801950: tensor([1, 2, 3, 4, 5])}

In [41]:
alice._objects

{}

In [42]:
# we transfered the data from bob to alice
x.move(alice)

(Wrapper)>[PointerTensor | me:31355857172 -> alice:31355857172]

In [43]:
bob._objects

{}

In [44]:
alice._objects

{31355857172: tensor([1, 2, 3, 4, 5])}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [61]:
bob._objects

{32433724544: tensor([1, 2, 3, 4, 5])}

In [62]:
alice._objects

{24940935406: (Wrapper)>[PointerTensor | alice:24940935406 -> bob:32433724544]}

In [63]:
# transfer data from bob to alice
x.remote_get()

(Wrapper)>[PointerTensor | me:56565578794 -> alice:24940935406]

In [64]:
bob._objects

{}

In [65]:
alice._objects

{24940935406: tensor([1, 2, 3, 4, 5])}

In [66]:
x.move(bob)

(Wrapper)>[PointerTensor | me:56565578794 -> bob:56565578794]

In [67]:
x

(Wrapper)>[PointerTensor | me:56565578794 -> bob:56565578794]

In [68]:
bob._objects

{56565578794: tensor([1, 2, 3, 4, 5])}

In [69]:
alice._objects

{}